In [3]:
def insta_searching(word: str) -> str:
    """인스타그램의 검색 결과 URL을 만드는 함수"""
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [57]:
from selenium import webdriver

import time

driver = webdriver.Chrome('C:/Users/gram/Downloads/chromedriver_win32/chromedriver.exe')

url = insta_searching('제주도맛집')
driver.get(url)
time.sleep(2)

In [59]:
# 인스타 로그인을 해야 게시글을 볼 수 있음

def insta_login(driver):
    with open("./user.txt", "r") as f:
        email = f.readline()
        password = f.readline()
        time.sleep(2)

        try:
            login = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
            login.click()
            time.sleep(2)
        except: # noqa
            pass

        facebook_login = driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF')
        facebook_login.click()
        time.sleep(2)

        input_id = driver.find_element_by_id('email')
        input_id.clear()
        input_id.send_keys(email)
        time.sleep(2)

        input_pw = driver.find_element_by_id('pass')
        input_pw.clear()
        input_pw.send_keys(password)
        input_pw.submit()
        time.sleep(5)
        
        try:
            account = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
            account.click()
            time.sleep(3)
        except:  # noqa
            pass

In [28]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(3)
    
select_first(driver)

In [63]:
from bs4 import BeautifulSoup

import re
import unicodedata

def get_content(driver):
    time.sleep(2)
    
    # 현재 페이지 HRML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except: # noqa
        content = ''

    # 본문에서 해시태그 가져오기
    tags = re.findall(r'#[^/s#,\\]+', content)

    # 작성일자 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]

    # 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except: # noqa
        like = 0

    # 조회 수 가져오기
    try:
        watch = soup.select('div.HbPOm._9Ytll > span')[0].text[3:-1]
    except: # noaq
        watch = 0

    # 위치 정보 가져오기
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except: # noqa
        place = ''

    # 수집한 정보 저장
    data = {'content': content, 'date': date, 'like': like, 'watch': watch, 'place': place, 'tags': tags}
    return data

get_content(driver)

{'content': '#맥도날드 #치킨텐더 #치즈스틱 #감튀',
 'date': '2021-01-03',
 'like': '15',
 'watch': 0,
 'place': '',
 'tags': ['#맥도날드 ', '#치킨텐더 ', '#치즈스틱 ', '#감튀']}

In [33]:
def move_to_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
move_to_next(driver)

In [75]:
# 인스타그램 크롤링 전체 과정

from selenium import webdriver
from bs4 import BeautifulSoup

import re
import time
import unicodedata

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Users/gram/Downloads/chromedriver_win32/chromedriver.exe')

# 인스타그램 접속
url = insta_searching('제주핫플')
driver.get(url)
time.sleep(2)

# 게시글 조회를 위한 인스타그램 로그인
insta_login(driver)

# 인스타그램 검색
url = insta_searching('제주핫플')
driver.get(url)
time.sleep(2)

# 첫 번째 게시글 조회
select_first(driver)

results = []

target = 200  # 크롤링할 게시글 수
for i in range(target):
    #
    print(i)
    #
    data = get_content(driver)
    results.append(data)
    move_to_next(driver)
    
print(results[:2])
print(len(results))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
[{'content': '<광고>추워진 몸을 녹이는 따끈한 국물을 위해 도톰한 사기그릇에 담아드리는 갈비국수!! 번거롭지만 손님들을 위해 끝까지 뜨끈뜨끈하게 드실수 있도록!!! 신경쓰는 #산도롱맨도롱▪️오픈시간: 08:00 - 20:00 (19:00까지 입장) ▪️전화: 064-782-5105▪️제주시 구좌읍 해맞이해안로 2284렌터카네비는 가끔 틀리실수 있으니 폰네비로 찾아오세요~~~ #월정리맛집 #성산일출봉맛집#제주도흑돼지맛집 #제주공항근처맛집 #성산맛집 #제주맛집 #제주시맛집#종달리맛집 #우도여행 #제주핫플#제주맛집 #제주도맛집 #제주맛집추천'

In [76]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'watch', 'place', 'tags']
results_df.to_excel('./files/crawling_jejuHotPlace.xlsx')